## Imports

In [164]:
import pandas as pd, seaborn as sns, numpy as np
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

---

## Funciones auxiliares

In [165]:
def quitar_coma(numero):
     return numero.replace(",", "")
    
def asociatividad(flag):
    asociatividades = {
        "--D1=32768,1,32" : 0,
        "--D1=32768,2,32" : 2,
        "--D1=32768,4,32" : 4,
        "--D1=32768,1024,32" : 1024
    }
    return asociatividades[flag]

def algoritmo_utilizado(ruta):
    if ruta == "src/mult/mmult_asm.S":
        return "mmult_asm"
    return "mmult_c"

def es_clear_cache(flag):
    if flag == "clear_cache=yes":
        return True
    return False

---

# Carga de datos

In [166]:
benchmark = pd.read_csv("merge.txt", header=None ,delim_whitespace=True)

---

## Inspección

In [167]:
benchmark.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,inputs/1.txt,clear_cache=no,src/mult/mmult_asm.S,"--I1=32768,4,32","--D1=32768,1,32",57,7,7,3,0,0,7,0,0,events,annotated
1,2,inputs/2.txt,clear_cache=no,src/mult/mmult_asm.S,"--I1=32768,4,32","--D1=32768,1,32",222,7,7,17,0,0,10,0,0,events,annotated
2,3,inputs/3.txt,clear_cache=no,src/mult/mmult_asm.S,"--I1=32768,4,32","--D1=32768,1,32",609,7,7,55,0,0,15,0,0,events,annotated


### Elimino columnas no relevantes

In [168]:
benchmark.drop(columns=[1, 4, 6, 7, 8, 11, 12, 13, 14, 15, 16], inplace=True)

In [169]:
benchmark.head(3)

,0,2,3,5,9,10
0,1,clear_cache=no,src/mult/mmult_asm.S,"--D1=32768,1,32",3,0
1,2,clear_cache=no,src/mult/mmult_asm.S,"--D1=32768,1,32",17,0
2,3,clear_cache=no,src/mult/mmult_asm.S,"--D1=32768,1,32",55,0


### Asigno nombres de columnas

In [170]:
benchmark.columns = ["dimension", "clear_cache", "funcion", "asociatividad",
                     "Dr", "D1mr"]

### Reemplazo valores

In [171]:
benchmark["clear_cache"] = benchmark["clear_cache"].map(es_clear_cache)
benchmark["funcion"] = benchmark["funcion"].map(algoritmo_utilizado)
benchmark["asociatividad"] = benchmark["asociatividad"].map(asociatividad)

### Convierto a número

In [172]:
for columna in ["Dr", "D1mr"]:
    benchmark[columna] = pd.to_numeric(benchmark[columna].map(quitar_coma))

In [173]:
benchmark.head(3)

,dimension,clear_cache,funcion,asociatividad,Dr,D1mr
0,1,False,mmult_asm,0,3,0
1,2,False,mmult_asm,0,17,0
2,3,False,mmult_asm,0,55,0


---

# Calculo missrates

In [174]:
benchmark["miss_rate"] = benchmark["D1mr"] / benchmark["Dr"]

In [175]:
benchmark.head(2)

,dimension,clear_cache,funcion,asociatividad,Dr,D1mr,miss_rate
0,1,False,mmult_asm,0,3,0,0.0
1,2,False,mmult_asm,0,17,0,0.0


---

# Ploteo

In [323]:
for funcion in ["mmult_asm", "mmult_c"]:
    for clear_cache in [True, False]:
        for asociatividad in [0, 2, 4, 1024]:
            loc = benchmark.loc[(benchmark["funcion"] == funcion) & (benchmark["clear_cache"] == clear_cache) & (benchmark["asociatividad"] == asociatividad)]
            fig = plt.figure(figsize=(10, 6))
            ax = sns.lineplot(x=loc["dimension"], y=loc["miss_rate"], color="g")
            ax.legend(loc='upper left', labels=["Miss rate"])
            ax2 = plt.twinx()
            sns.lineplot(ax=ax2, x=loc["dimension"], y=loc["D1mr"])
            ax2.legend(loc='upper right', labels=['Miss count'])            
            label = "Funcion: {}, Clear_cache={}, Asociatividad={}".format(funcion, str(clear_cache), asociatividad)
            plt.title(label=label)
            plt.savefig("figures/{}_{}_{}.png".format(funcion, str(clear_cache), asociatividad))
            plt.close(fig)
            

### Ploteo la cantidad de accesos

In [326]:
accesos = benchmark.loc[(benchmark["funcion"] == "mmult_asm") & (benchmark["clear_cache"] == True) & (benchmark["asociatividad"] == 0)]
fig = plt.figure(figsize=(10, 6))
ax = sns.lineplot(x=accesos["dimension"], y=accesos["Dr"], color="orange")
plt.title(label="Cantidad de accesos a memoria según la dimensión")
plt.savefig("figures/accesos.png")
plt.close(fig)

---

# Cálculo Teórico

### Carga de datos

In [313]:
DM = pd.read_csv("DM.csv", header=None)
TWO_WAY = pd.read_csv("2WAY.csv", header=None)
FOUR_WAY = pd.read_csv("4WAY.csv", header=None)
todos = [DM, TWO_WAY, FOUR_WAY]

### Nombro columnas y calculo missrate

In [314]:
for asociatividad in todos:
    asociatividad.columns = ["dimension", "asociatividad", "hits", "misses"]
    asociatividad["accesos"] = asociatividad["hits"] + asociatividad["misses"]
    asociatividad["miss_rate"] = asociatividad["misses"] / asociatividad["accesos"]

### Mapeo Directo

In [320]:
nombres = ["Mapeo Directo", "2-Way Associative", "4-Way Associative"]
i = 0
for asociatividad in todos:
    medias = asociatividad.groupby("dimension")["miss_rate"].agg("mean")
    medias = medias.to_frame().reset_index()
    f, ax = plt.subplots(figsize=(10, 7))
    sns.despine(bottom=True, left=True)
    sns.stripplot(x="dimension", y="miss_rate", data=asociatividad, alpha=.25, zorder=1)
    sns.pointplot(x="dimension", y="miss_rate", data=asociatividad, dodge=.532, join=False, palette="dark",
              markers="d", scale=.75, ci=None)
    sns.lineplot(x=[0, 1, 2, 3, 4, 5, 6, 7], y="miss_rate", data=medias)
    plt.title(label="Distribución de Miss rate para caché {}".format(nombres[i]))
    plt.savefig("figures/{}.png".format(nombres[i]))
    plt.close(f)
    i +=1

---